---
author: "Harvey"
title: "Building a Quarto Extension"
date: "2023-11-20"
description: "First attempt at building a Quarto extension"
categories: ["Quarto"]
---


# Building my First Quarto Shortcode Extension

[Quarto](https://quarto.org/) is a scientific publishing system used to build documents, presentations, books and websites.  Quarto extensions are a powerful way to extend the behavior of Quarto.  These extensions are built using Lua, a language that I do not have much familiarity with but, building a simple extension is actually quite straighforward.

In this example I'll create a shortcode extension that allows you to add comments to a Quarto document.  The comments can be turned on or off by using an environemnt variable defined in 

## Initialization

To initialize a Quarto shortcode extension, use the following command and enter a name for the extension (in this case *comment*)

```bash
quarto create extension shortcode
```

The following folder structure will be created:

```
comment
├── _extensions  
│   └── comment  
│       ├── _extension.yml  
│       └── comment.lua  
├── .gitignore  
├── .luarc.json  
├── example.qmd
└── README.md  
```

I use VS Code for coding.  When the `quarto create` command is executed, VS Code opens a new window at the `comment` folder, ready for development.

## Building the Shortcode Extension

This Quarto extension allows an author to add comments to a Quarto document.  It also adds a parameter to control if comments should included on not when a file is rendered.  I've found this extension useful when adding instructional comments to a template.

The extension contains three shortcodes.  One to start a comment block, one to end a comment block and one to add an inline comment.  
The code concept is pretty simple.  For comment blocks, surround the contents in a div with either the `cmt` class (which can be used to control styling) when the comment variable is set to true, or `display: none` when the comment variable is set to false.  For inline comments surround the contents in a span with the `cmt` class when the comment variable is set to true.

The `comment.lua` file is shown below.

```lua
-- start a comment block
function comment_start(args, kwargs, meta)
  cmt = meta['show_comments']
  if cmt == nil or cmt == false then
    return pandoc.RawBlock('html', "<div style = 'display: none;'>")
  else
    return pandoc.RawBlock('html', "<div class = 'cmt'>")
  end
end

-- end a comment block
function comment_stop()
  return pandoc.RawBlock('html', "</div>")
end

-- inline comment
function comment(args, kwargs, meta)
  txt = pandoc.utils.stringify(args[1])
  cmt = meta['show_comments']
  if cmt == true then
    return pandoc.RawInline('html', "<span class = 'cmt'>" .. txt .. "</span>")
  end 
end
```

In addition to the `comment.lua` file, an `example.qmd` should be written (plus, in this case, a `style.css` css file).  If the `example.qmd` file is previewed then it will automatically update as you edit comment.lua - a great way to ensure that your extension is working as expected.  The `example.qmd` and `style.css` files for the comment extension are shown below:

```md
---
title: "Comment Example"
format:
  html:
    css: style.css
show_comments: true
---

## Comments Test - Commented text within a paragraph

This is some uncommented text.
{{</* comment_start */>}}
Here is a <b>comment</b> containing some instructional information.
{{</* comment_stop */>}}
Finally, some additional uncommented text.

## Comments Test - Inline commenting

This comment is an inline comment {{</* comment "It can include instruction within the text" */>}} followed by addititional text.
```


```{css}
#| eval: false
#| title: style.css
.cmt {
  color: #AAAA00
}
```